### Import Required Libraries

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import spacy
from tqdm import tqdm

pd.set_option('display.max_colwidth', 200)

In [2]:
!pip install tables

### Read Dataset

In [3]:
# read data
df_questions = pd.read_hdf('auto_tagging_data_v2.h5')

In [4]:
df_questions.head()

,Id,Title,Body,Tags
0,6,The Two Cultures: statistics vs. machine learning?,"<p>Last year, I read a blog post from <a href=""http://anyall.org/"">Brendan O'Connor</a> entitled <a href=""http://anyall.org/blog/2008/12/statistics-vs-machine-learning-fight/"">""Statistics vs. Mach...",[machine-learning]
1,21,Forecasting demographic census,<p>What are some of the ways to forecast demographic census with some validation and calibration techniques?</p>\n\n<p>Some of the concerns:</p>\n\n<ul>\n<li>Census blocks vary in sizes as rural\n...,[forecasting]
2,22,Bayesian and frequentist reasoning in plain English,<p>How would you describe in plain English the characteristics that distinguish Bayesian from Frequentist reasoning?</p>\n,[bayesian]
3,31,What is the meaning of p values and t values in statistical tests?,"<p>After taking a statistics course and then trying to help fellow students, I noticed one subject that inspires much head-desk banging is interpreting the results of statistical hypothesis tests....","[hypothesis-testing, t-test, p-value, interpretation]"
4,36,Examples for teaching: Correlation does not mean causation,"<p>There is an old saying: ""Correlation does not mean causation"". When I teach, I tend to use the following standard examples to illustrate this point:</p>\n\n<ol>\n<li>number of storks and birth ...",[correlation]


In [5]:
# combine title and body
df_questions['Text'] = df_questions["Title"] + " " + df_questions["Body"]

### Clean and Pre-process Data

In [6]:
def clean_text(text):
    # remove html tags and url links
    text = re.sub(r'<.*?>', '', text)
    # remove everything alphabets
    text = re.sub("[^a-zA-Z]"," ",text)
    # remove whitespaces
    text = ' '.join(text.split())
    
    return text

In [7]:
df_questions['Text'] = df_questions['Text'].apply(lambda x: clean_text(x))
df_questions['Text'] = df_questions['Text'].str.lower()

In [8]:
df_questions[['Id', 'Text', 'Tags']].sample(5)

,Id,Text,Tags
49539,95407,how to create a dependent variable in regression when it does not exist in data in ques i have factors which could affect purchasing decision now i wanted to find the co relation between factors w...,"[regression, correlation, chi-squared]"
50515,79311,repeated measures glm restricted permutations i have a question regarding repeated measures and glms suppose i have counted the abundance of some species in lakes at different time points every la...,"[r, repeated-measures, generalized-linear-model]"
39436,37832,decomposition of the sum of two random variables we observe the input samples as z x y where distribution of z could be estimated by histogram of samples and x y are two independent random variabl...,[probability]
8003,26951,given a two populations each split by categories how do i test the hypothesis that the mean is larger in the second population the means for each category varies but my hypothesis is that the seco...,"[hypothesis-testing, anova, mean]"
64076,77270,feature selection and cross validation i m working on a project and i would like to know if the following strategy is good correct sorry if this is a basic stupid idea i m new to this the input is...,"[machine-learning, data-mining, svm, cross-validation, feature-selection]"


### Encode Text to Numbers

In [9]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [10]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_questions['Text'])

In [11]:
# check unique words count
len(tokenizer.word_index)

81956

In [12]:
# check unique words count
vocab_size = len(tokenizer.word_index) + 1
vocab_size

81957

In [13]:
sequences = tokenizer.texts_to_sequences(df_questions['Text'])

In [14]:
i = 0
print(df_questions['Text'][i], '\n'), print(sequences[i])

the two cultures statistics vs machine learning last year i read a blog post from brendan o connor entitled statistics vs machine learning fight that discussed some of the differences between the two fields andrew gelman responded favorably to this simon blomberg from r s fortunes package to paraphrase provocatively machine learning is statistics minus any checking of models and assumptions brian d ripley about the difference between machine learning and statistics user vienna may season s greetings andrew gelman in that case maybe we should get rid of checking of models and assumptions more often then maybe we d be able to solve some of the problems that the machine learning people can solve but we can t there was also the statistical modeling the two cultures paper by leo breiman in which argued that statisticians rely too heavily on data modeling and that machine learning techniques are making progress by instead relying on the predictive accuracy of models has the statistics field 

(None, None)

__Find right maximum length for the sequences__

In [15]:
seq_lengths = []

for i in sequences:
    seq_lengths.append(len(i))

In [16]:
print("30th percentile: ", pd.Series(seq_lengths).quantile(0.3))
print("40th percentile: ", pd.Series(seq_lengths).quantile(0.4))
print("50th percentile: ", pd.Series(seq_lengths).quantile(0.5))
print("60th percentile: ", pd.Series(seq_lengths).quantile(0.6))
print("70th percentile: ", pd.Series(seq_lengths).quantile(0.7))
print("80th percentile: ", pd.Series(seq_lengths).quantile(0.8))
print("90th percentile: ", pd.Series(seq_lengths).quantile(0.9))
print("95th percentile: ", pd.Series(seq_lengths).quantile(0.95))
print("99th percentile: ", pd.Series(seq_lengths).quantile(0.99))

30th percentile:  97.0
40th percentile:  116.0
50th percentile:  137.0
60th percentile:  162.0
70th percentile:  193.0
80th percentile:  238.0
90th percentile:  320.0
95th percentile:  411.0
99th percentile:  678.0


In [17]:
max_length = 125

# padding
padded_seq = pad_sequences(sequences, maxlen=max_length)

### Reshape Target Variable

In [18]:
from sklearn.preprocessing import MultiLabelBinarizer

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(df_questions['Tags'])
y = multilabel_binarizer.transform(df_questions['Tags'])

In [19]:
padded_seq.shape, y.shape

((76365, 125), (76365, 100))

### Train-Test Split

In [20]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(padded_seq, y, 
                                                    test_size=0.2, 
                                                    random_state=9)

### Model Building

In [21]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Embedding, GlobalMaxPool1D, Dropout, Conv1D
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [22]:
model = Sequential()
model.add(Embedding(2000, 128, input_length = max_length))
model.add(Dropout(0.15))
model.add(Conv1D(300, 3, padding = 'valid', activation = "relu", strides = 1))
model.add(GlobalMaxPool1D())
model.add(Dense(100, activation = "sigmoid"))
#model.add(Activation('sigmoid'))

In [23]:
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 125, 128)          256000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 125, 128)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 123, 300)          115500    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 300)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
Total params: 401,600
Trainable params: 401,600
Non-trainable params: 0
_________________________________________________________________


In [24]:
callbacks = [
             EarlyStopping(patience=3),
             ModelCheckpoint(filepath='model-conv1d_v1.h5', save_best_only=True)
            ]

In [26]:
# train model
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=512,
                    validation_split=0.1,
                    callbacks=callbacks)

### Predictions and Performane Evaluation

In [ ]:
# use the code below to load the saved model
# model = load_model('model-conv1d_v1.h5')

In [ ]:
preds = model.predict(x_val)

In [ ]:
preds.shape

In [ ]:
# set threshold to 0.45
preds_int = (preds >= 0.45).astype(int)

In [ ]:
from sklearn.metrics import f1_score

# calculate F1 score
f1_score(y_val, preds_int, average="micro")

### Inference

In [ ]:
# model = load_model('model-conv1d_v1.h5')

In [ ]:
def infer_tags(q):
    q = clean_text(q)
    q = q.lower()
    q_seq = tokenizer.texts_to_sequences([q])
    q_seq_padded = pad_sequences(q_seq, maxlen=300)
    q_pred = model.predict(q_seq_padded)
    q_pred = (q_pred >= 0.3).astype(int)
    
    return multilabel_binarizer.inverse_transform(q_pred)

In [ ]:
# give new question
new_q = "Regression line in ggplot doesn't match computed regression Im using R and created a chart using ggplot2. I then create a regression so I can make some predicitions I pass my data frame of to the predict function predict(regression, Measures) I'd expect the predictions to be the same as if I used the regression line on the chart, but they aren't the same. Why would this be the case? Is there a setting in ggplot or is my expectation incorrect?"

# get tags
infer_tags(new_q)